# Installation testing notebook

Run the following cells to check that the installation was successful ✅

## Testing pynguin

See pynguin documentation for [more](https://pynguin.readthedocs.io/en/latest/api.html) 🐧

After running the cell, look in `pynguin_results/` for [test_example.py](pynguin_results/test_example.py) to see the generated test cases.

In [ ]:
import os
import sys
import subprocess

# 1) Prepare a results directory
os.makedirs("pynguin_results", exist_ok=True)

# 2) Use Pynguin’s programmatic API
from pynguin.configuration import (
    Configuration,
    TestCaseOutputConfiguration,
    ExportStrategy
)
from pynguin.generator import set_configuration, run_pynguin

# Build the minimal test-case-output config
output_config = TestCaseOutputConfiguration(
    export_strategy=ExportStrategy.PY_TEST,   # Use pytest style
    format_with_black=True,                   # Auto-format with Black
    output_path="pynguin_results"             # Where to write tests
)

# Build the main configuration
cfg = Configuration(
    project_path=".",       # Path where example.py lives
    module_name="example",  # The module to test (i.e., example.py)
    test_case_output=output_config
)

# “Activate” it and run
set_configuration(cfg)
rc = run_pynguin()
print("Pynguin return code:", rc)

# 3) Run coverage + pytest in one go
cmd = [
    sys.executable, "-m", "coverage", "run",
    "--source=example",       # only measure example.py
    "-m", "pytest", "pynguin_results",
    "-q"                      # quiet pytest (dots only)
]
print("▶ Running:", " ".join(cmd))
proc = subprocess.run(cmd, capture_output=True, text=True)

# 4) Always print out what pytest saw
print(proc.stdout)
if proc.returncode != 0:
    print("─── STDERR ───", file=sys.stderr)
    print(proc.stderr, file=sys.stderr)
    raise RuntimeError(f"pytest failed with exit code {proc.returncode}")

# 5) Tests passed, now show coverage report
print("\nTests passed. Generating coverage report…\n")

# 6) Terminal report (with missing lines)
subprocess.run(
    [sys.executable, "-m", "coverage", "report", "-m"],
    check=True
)

# 7) HTML report
html_dir = "htmlcov"
subprocess.run(
    [sys.executable, "-m", "coverage", "html", "-d", html_dir],
    check=True
)

In [ ]:
import webbrowser

# 8) Automatically open coverage report in browser
abs_path = os.path.abspath(os.path.join(html_dir, "index.html"))
file_url = f"file://{abs_path}"
print(f"\nFile URL:\n{file_url}")
webbrowser.open(file_url)